In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
import seaborn as sns
from matplotlib import pyplot as plt
import sklearn.cross_validation
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
#because dataset is skewed we can try out logistic regression with balanced weights
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc , roc_auc_score, confusion_matrix, mean_absolute_error
from sklearn.tree import DecisionTreeClassifier , export_graphviz
from sklearn.cross_validation import train_test_split
from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn import tree

/Library/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
%pylab inline

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.cross_validation import train_test_split

try:
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.pipeline import make_pipeline
except ImportError:
    # use backports for sklearn 1.4
    # available from https://s3.amazonaws.com/datarobotblog/notebooks/sklearn_backports.py
    from sklearn_backports import PolynomialFeatures
    from sklearn_backports import make_pipeline

# ignore DeprecateWarnings by sklearn
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

Populating the interactive namespace from numpy and matplotlib


In [3]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(df_train.SeriousDlqin2yrs))
    plt.xticks(tick_marks, df_train.SeriousDlqin2yrs, rotation=45)
    plt.yticks(tick_marks, df_train.SeriousDlqin2yrs)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [4]:
df_train=pd.read_csv('train_ireg_amd_dmd.csv')
df_test=pd.read_csv('test_ireg_amd_dmd.csv')

In [5]:
df_train.head(10)

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0,1,0.766127,45,2,0.802982,9120.000000,13,0,6,0,2.0
1,1,0,0.957151,40,0,0.121876,2600.000000,4,0,0,0,1.0
2,2,0,0.658180,38,1,0.085113,3042.000000,2,1,0,0,0.0
3,3,0,0.233810,30,0,0.036050,3300.000000,5,0,0,0,0.0
4,4,0,0.907239,49,1,0.024926,63588.000000,7,0,1,0,0.0
5,5,0,0.213179,74,0,0.375607,3500.000000,3,0,1,0,1.0
6,6,0,0.305682,57,0,5710.000000,8738.957438,8,0,3,0,0.0
7,7,0,0.754464,39,0,0.209940,3500.000000,8,0,0,0,0.0
8,8,0,0.116951,27,0,46.000000,3045.627787,2,0,0,0,1.0
9,9,0,0.189169,57,0,0.606291,23684.000000,9,0,4,0,2.0


In [6]:
#Currently getting subset of data and just wrangling around and trying

Y_train=np.array(df_train.SeriousDlqin2yrs)
#features = ['RevolvingUtilizationOfUnsecuredLines', 'DebtRatio',
#            'MonthlyIncome', 'age', 'NumberOfTimes90DaysLate']
features=['RevolvingUtilizationOfUnsecuredLines','age','NumberOfTime30-59DaysPastDueNotWorse','DebtRatio','MonthlyIncome','NumberOfOpenCreditLinesAndLoans','NumberOfTimes90DaysLate','NumberRealEstateLoansOrLines','NumberOfTime60-89DaysPastDueNotWorse','NumberOfDependents']
X_train=np.array(df_train[features])

In [7]:
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size=0.25)

In [65]:
#just checking
model_3 = DecisionTreeClassifier(max_depth=3,class_weight='auto')

In [66]:
#according to library samples are equally weighted, it also needs to be checked
model_3.fit(X_train,Y_train)

DecisionTreeClassifier(class_weight='auto', criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')

In [11]:
#dotfile = open('treemodel_depth_3.dot', 'w')
#export_graphviz(model_3,out_file=dotfile, feature_names=features)
#dotfile.close()
#!dot -Tpng treemodel_depth_3.dot -o treemodel_depth_3.png
a=Image('treemodel_depth_3.png',unconfined=True)
#save(a.data)


In [43]:
from IPython.display import Image  
dot_data = StringIO()  
tree.export_graphviz(model_3, out_file=dot_data,  feature_names=features)  
graph = pydot.graph_from_dot_data(dot_data.getvalue())  
 
import pydotplus
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
#graph.write_pdf("decisioncredit.pdf")
#print graph.create_jpeg('a.jpg')
#Image(graph.create_png()) 
#for visualization

In [67]:
pclass_3=model_3.predict(X_validation)

In [68]:
print Y_validation
print np.count_nonzero(pclass_3==Y_validation)/float(len(pclass_3))
print "higher accuracy dont mean that classifier, lets also compare it with probability and then checking confusion matrix"
prob_3=model_3.predict_proba(X_validation)
print prob_3 # 0 is assigned if on 1st column prob is higher

[0 0 1 ..., 0 0 0]
0.68472
higher accuracy dont mean that classifier, lets also compare it with probability and then checking confusion matrix
[[ 0.45051814  0.54948186]
 [ 0.81513814  0.18486186]
 [ 0.19502138  0.80497862]
 ..., 
 [ 0.45051814  0.54948186]
 [ 0.45051814  0.54948186]
 [ 0.81513814  0.18486186]]


In [70]:
print "ROC score on model with depth3 =",roc_auc_score(Y_validation,prob_3[:,1])

ROC score on model with depth3 = 0.816456130523


In [71]:
print "Confusion Matrix"
cm = confusion_matrix(Y_validation,pclass_3)
print(cm)
print classification_report(np.array(Y_validation),np.array(pclass_3))
print "interpretation: remaining"

Confusion Matrix
[[23543 11424]
 [  399  2134]]
             precision    recall  f1-score   support

          0       0.98      0.67      0.80     34967
          1       0.16      0.84      0.27      2533

avg / total       0.93      0.68      0.76     37500

interpretation: remaining


In [73]:
model_full = DecisionTreeClassifier(class_weight='auto')
model_full.fit(X_train,Y_train)

DecisionTreeClassifier(class_weight='auto', criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')

In [74]:
pclass_full=model_full.predict(X_validation)
print Y_validation
print np.count_nonzero(pclass_full==Y_validation)/float(len(pclass_full))
print "higher accuracy dont mean that classifier, lets also compare it with probability and then checking confusion matrix"
prob_full=model_full.predict_proba(X_validation)
print prob_full # 0 is assigned if on 1st column prob is higher

[0 0 1 ..., 0 0 0]
0.902746666667
higher accuracy dont mean that classifier, lets also compare it with probability and then checking confusion matrix
[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 ..., 
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]


In [75]:
print "Confusion Matrix"
cm = confusion_matrix(Y_validation,pclass_full)
print(cm)
print classification_report(np.array(Y_validation),np.array(pclass_full))
print "interpretation: remaining"

Confusion Matrix
[[33238  1729]
 [ 1918   615]]
             precision    recall  f1-score   support

          0       0.95      0.95      0.95     34967
          1       0.26      0.24      0.25      2533

avg / total       0.90      0.90      0.90     37500

interpretation: remaining


In [77]:
print "ROC score on model with depth full =",roc_auc_score(Y_validation,prob_full[:,1])

ROC score on model with depth full = 0.596700610313


In [81]:
#lets use cross validation 
modeldepth=[1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
besttree=None
bestscore=0
bestdepth=0
for d in modeldepth:
    model = DecisionTreeClassifier(class_weight='auto',max_depth=d)
    model.fit(X_train,Y_train)
    prob=model.predict_proba(X_validation)
    auc_score=roc_auc_score(Y_validation,prob[:,1])
    if(auc_score>bestscore):
        bestscore=auc_score
        besttree=model
        bestdepth=d
    print "ROC score on model with depth  ",d,' :',roc_auc_score(Y_validation,prob[:,1])
    
print "best model with depth: ",bestdepth , " score: ",bestscore 

#plotting of every tree on one graph to be done
#identifying other validaiton technique for example classification error, K FOld Cross Validation and other techniques

ROC score on model with depth   1  : 0.728162939619
ROC score on model with depth   3  : 0.816456130523
ROC score on model with depth   4  : 0.830862065639
ROC score on model with depth   5  : 0.844222403773
ROC score on model with depth   6  : 0.844932181333
ROC score on model with depth   7  : 0.84331213827
ROC score on model with depth   8  : 0.838209955806
ROC score on model with depth   9  : 0.825048626582
ROC score on model with depth   10  : 0.811281193206
ROC score on model with depth   11  : 0.792674461289
ROC score on model with depth   12  : 0.770574655291
ROC score on model with depth   13  : 0.750370861767
ROC score on model with depth   14  : 0.726692182876
ROC score on model with depth   15  : 0.705470645601
ROC score on model with depth   16  : 0.689532427117
ROC score on model with depth   17  : 0.671231363809
ROC score on model with depth   18  : 0.667135318641
ROC score on model with depth   19  : 0.665368100549
ROC score on model with depth   20  : 0.652689252066
be

In [85]:
# lets use best model to learn whole model
X_train_final=np.array(df_train[features])
Y_train_final=np.array(df_train.SeriousDlqin2yrs)
#print X_train_final

model = DecisionTreeClassifier(class_weight='auto',max_depth=bestdepth)
finalmodel=model.fit(X_train_final,Y_train_final)
features=['RevolvingUtilizationOfUnsecuredLines','age','NumberOfTime30-59DaysPastDueNotWorse','DebtRatio','MonthlyIncome','NumberOfOpenCreditLinesAndLoans','NumberOfTimes90DaysLate','NumberRealEstateLoansOrLines','NumberOfTime60-89DaysPastDueNotWorse','NumberOfDependents']
X_test=np.array(df_test[features])

pred_test=finalmodel.predict_proba(X_test)[:,1]

In [86]:
idx=np.arange(1,len(pred_test)+1,1)
print len(pred_test)

#output=np.array()
#output=np.zeros((len(pred_test),2))
#output[:,0]=idx
#output[:,1]=pred_test
output=pd.DataFrame(pred_test)
dfoutput=pd.DataFrame(output)
dfoutput['Id']=idx
dfoutput['Probability']=dfoutput[0]
del dfoutput[0]
dfoutput

101503


,Id,Probability
0,1,0.604593
1,2,0.535548
2,3,0.114514
3,4,0.506715
4,5,0.604593
5,6,0.437565
6,7,0.481211
7,8,0.348413
8,9,0.114514
9,10,0.933445


In [87]:
dfoutput.to_csv('pred3_tree.csv',index=False)